- Hi ppl, use this notebook to verify/solve your answers.
- You can connect with me on LinkedIn: https://www.linkedin.com/in/akshit12mittal
- PS- I didn't solve any ques, I hacked ga5 but in case u want to understand things, here's the colab.

# Q1
1. Run the code.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import files

# Step 1: Upload Excel File
uploaded = files.upload()  # Manually upload the file
file_name = list(uploaded.keys())[0]  # Get the uploaded filename

# Step 2: Load Data
df = pd.read_excel(file_name)

# Step 3: Trim and Normalize Country Names
df['Country'] = df['Country'].str.strip().str.upper()  # Remove spaces and convert to uppercase

# Step 4: Standardize Country Names
country_mapping = {
    "USA": "US", "U.S.A": "US", "UNITED STATES": "US", "US": "US",
    "BRA": "BR", "BRAZIL": "BR", "BR": "BR",
    "U.K": "GB", "UK": "GB", "UNITED KINGDOM": "GB",
    "FR": "FRANCE", "FRA": "FRANCE", "FRANCE": "FRANCE",
    "IND": "INDIA", "IN": "INDIA", "INDIA": "INDIA",
    "AE": "UAE", "U.A.E": "UAE", "UNITED ARAB EMIRATES": "UAE", "UAE": "UAE"
}
df['Country'] = df['Country'].replace(country_mapping)

# Step 5: Standardize Date Formats
def convert_to_datetime(date):
    for fmt in ("%m-%d-%Y", "%Y/%m/%d", "%Y-%m-%d"):
        try:
            return datetime.strptime(str(date), fmt)
        except ValueError:
            continue
    return pd.NaT  # Return NaT for unrecognized formats

df['Date'] = df['Date'].apply(convert_to_datetime)

# Step 6: Extract Product Name (Before "/")
df['Product/Code'] = df['Product/Code'].str.split('/').str[0].str.strip()

# Step 7: Clean and Convert Sales & Cost Columns
df['Sales'] = df['Sales'].astype(str).str.replace("USD", "").str.strip().astype(float)
df['Cost'] = df['Cost'].astype(str).str.replace("USD", "").str.strip()

# Handle missing cost values (assume 50% of Sales)
df['Cost'] = pd.to_numeric(df['Cost'], errors='coerce')
df['Cost'].fillna(df['Sales'] * 0.5, inplace=True)

# Step 8: Apply Filters
date_filter = datetime(2023, 1, 3, 16, 56, 57)  # Tue Jan 03 2023 16:56:57 GMT+0530
df_filtered = df[
    (df['Date'] <= date_filter) &
    (df['Product/Code'].str.lower() == "eta") &
    (df['Country'] == "BR")
]

# Step 9: Calculate Total Margin
total_sales = df_filtered['Sales'].sum()
total_cost = df_filtered['Cost'].sum()

if total_sales > 0:
    total_margin = (total_sales - total_cost) / total_sales
else:
    total_margin = 0

# Step 10: Display Results
print(f"Total Sales: {total_sales:.2f}")
print(f"Total Cost: {total_cost:.2f}")
print(f"Total Margin: {total_margin:.4f}")

# Step 11: Save Cleaned Data (Optional)
#df_filtered.to_csv("cleaned_data.csv", index=False)
#files.download("cleaned_data.csv")

Initial data shape: (1000, 7)
Unique countries after mapping: ['BR', 'Bra', 'Brazil', 'FRANCE', 'Fra', 'France', 'GB', 'INDIA', 'Ind', 'India', 'UAE', 'US', 'United Arab Emirates', 'United Kingdom', 'United States']
Unique products after extraction: ['Alpha', 'Beta', 'Delta', 'Epsilon', 'Eta', 'Gamma', 'Iota', 'Kappa', 'Theta', 'Zeta']


# Q2
1.  Run the code.

In [ ]:
from google.colab import files
import re

# Upload the text file
uploaded = files.upload()

# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Data Extraction: Read file line by line and extract student IDs
student_ids = set()
with open(file_name, 'r', encoding='utf-8') as file:
    for line in file:
        matches = re.findall(r'\b[A-Z0-9]{10}\b', line)  # Match exact 10-character alphanumeric IDs
        student_ids.update(matches)

# Reporting: Count the number of unique student IDs
print(f"Number of unique students: {len(student_ids)}")

# Q3
1.  Questions might be different, change things accordingly.

In [ ]:
import pandas as pd
import gzip
import re
import os
from datetime import datetime
import hashlib
from google.colab import files

# Function to compute SHA-256 hash
def compute_hash(text):
    return hashlib.sha256(text.encode()).hexdigest()

# Function to parse Apache log entries
def parse_log_line(line):
    log_pattern = (r'^(\S+) (\S+) (\S+) \[(.*?)\] "(\S+) (.*?) (\S+)" (\d+) (\S+) "(.*?)" "(.*?)" (\S+) (\S+)$')
    match = re.match(log_pattern, line)
    if match:
        return {
            "ip": match.group(1),
            "time": match.group(4),
            "method": match.group(5),
            "url": match.group(6),
            "protocol": match.group(7),
            "status": int(match.group(8)),
            "size": match.group(9),
            "referer": match.group(10),
            "user_agent": match.group(11),
            "vhost": match.group(12),
            "server": match.group(13)
        }
    return None

# Upload file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get uploaded file name

# Load and parse the log file
def load_logs(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' not found.")
        return pd.DataFrame()

    parsed_logs = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            parsed_entry = parse_log_line(line)
            if parsed_entry:
                parsed_logs.append(parsed_entry)
    return pd.DataFrame(parsed_logs)

# Convert time format
def convert_time(timestamp):
    return datetime.strptime(timestamp, "%d/%b/%Y:%H:%M:%S %z")

df = load_logs(file_path)

if not df.empty:
    df["datetime"] = df["time"].apply(convert_time)
    df["day_of_week"] = df["datetime"].dt.strftime('%A')
    df["hour"] = df["datetime"].dt.hour

    # Filter conditions
    filtered_df = df[
        (df["method"] == "GET") &
        (df["url"].str.startswith("/telugu/")) &
        (df["status"] >= 200) & (df["status"] < 300) &
        (df["day_of_week"] == "Sunday") &
        (df["hour"] >= 5) &
        (df["hour"] < 10)
    ]

    # Compute hash of the result
    result_hash = compute_hash(str(len(filtered_df)))

    # Output the count and hash
    print("Successful GET requests for /telugu/ on Sundays (5:00-9:59 AM):", len(filtered_df))
else:
    print("No log data available for processing.")

# Q4
1.  Questions might be different, change things accordingly.

In [ ]:
import pandas as pd
import gzip
import re
import os
from datetime import datetime
from google.colab import files

# Upload file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get uploaded file name

# Function to parse Apache log entries
def parse_log_line(line):
    log_pattern = (r'^(\S+) (\S+) (\S+) \[(.*?)\] "(\S+) (.*?) (\S+)" (\d+) (\S+) "(.*?)" "(.*?)" (\S+) (\S+)$')
    match = re.match(log_pattern, line)
    if match:
        return {
            "ip": match.group(1),
            "time": match.group(4),
            "method": match.group(5),
            "url": match.group(6),
            "protocol": match.group(7),
            "status": int(match.group(8)),
            "size": int(match.group(9)) if match.group(9).isdigit() else 0,
            "referer": match.group(10),
            "user_agent": match.group(11),
            "vhost": match.group(12),
            "server": match.group(13)
        }
    return None

# Load and parse the log file
def load_logs(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' not found.")
        return pd.DataFrame()

    parsed_logs = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            parsed_entry = parse_log_line(line)
            if parsed_entry:
                parsed_logs.append(parsed_entry)
    return pd.DataFrame(parsed_logs)

# Convert time format
def convert_time(timestamp):
    return datetime.strptime(timestamp, "%d/%b/%Y:%H:%M:%S %z")

df = load_logs(file_path)

if not df.empty:
    df["datetime"] = df["time"].apply(convert_time)
    df["date"] = df["datetime"].dt.strftime('%Y-%m-%d')

    # Filter conditions for /hindimp3/ on 2024-05-16
    filtered_df = df[
        (df["url"].str.startswith("/hindimp3/")) &
        (df["date"] == "2024-05-16")
    ]

    # Aggregate data by IP
    ip_data = filtered_df.groupby("ip")["size"].sum().reset_index()

    # Identify the top data consumer
    top_ip = ip_data.loc[ip_data["size"].idxmax()]

    # Output the result
    print(f"Top IP Address: {top_ip['ip']}")
    print(f"Total Bytes Downloaded: {top_ip['size']}")
else:
    print("No log data available for processing.")

# Q5
1. run below code and change city as per ur requirement in known_cities.
2. change product and minimum qty.

In [ ]:
!pip install fuzzywuzzy python-Levenshtein

import pandas as pd
from fuzzywuzzy import fuzz, process
from google.colab import files

# Upload the JSON file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get the uploaded file name
df = pd.read_json(file_path)

# List of known city names (add more as needed)
known_cities = ["Jakarta"]

def cluster_cities(city_series):
    """Standardize city names using fuzzy matching."""
    city_series = city_series.fillna('Unknown')

    def get_closest_city(city_name):
        best_match = process.extractOne(city_name, known_cities, scorer=fuzz.token_set_ratio)
        if best_match and best_match[1] > 90:  # Adjust the threshold as needed
            return best_match[0]
        return city_name

    city_series = city_series.apply(get_closest_city)
    return city_series

# Standardize city names
df['city'] = cluster_cities(df['city'])

# Debug: Check unique city names after clustering
print("Unique cities after clustering:", df['city'].unique())

# Filter sales for Shoes with at least 32 units
df_filtered = df[(df['product'] == "Shoes") & (df['sales'] >= 32)]

# Aggregate sales by city
df_grouped = df_filtered.groupby("city")["sales"].sum().reset_index()

# Identify the top-performing city
top_city = df_grouped.sort_values(by="sales", ascending=False).iloc[0]

# Find sales for Jakarta
jakarta_sales = df_grouped[df_grouped["city"].str.lower() == "jakarta"]["sales"].sum()

print(f"Total Shoes units sold in Jakarta (with at least 32 units per transaction): {jakarta_sales}")

# Q6
1.

# Q7
1.

# Q8
1.  Questions might be different, change things accordingly.

In [ ]:
SELECT post_id
FROM (
    SELECT post_id
    FROM (
        SELECT post_id,
               json_extract(comments, '$[*].stars.useful') AS useful_stars
        FROM social_media
        WHERE timestamp >= '2025-01-10T01:12:43.572Z'
    )
    WHERE EXISTS (
        SELECT 1 FROM UNNEST(useful_stars) AS t(value)
        WHERE CAST(value AS INTEGER) >= 4
    )
)
ORDER BY post_id;

#Q9
1. download your video and upload in colab file as videoplaback.mp4
2. change your times as per question in code
3. text received from here need improvement use chatgpt and tell chatgpt to add full stop and comma in given text.

In [ ]:
from google.colab import files
from pydub import AudioSegment
import speech_recognition as sr
import io
from moviepy.editor import VideoFileClip

def extract_audio_from_video(video_path, start_sec, end_sec, output_audio_path="temp_audio.wav"):
    """Extracts audio from a video segment and saves it as a WAV file."""
    with VideoFileClip(video_path) as video:
      audio_clip = video.audio.subclip(start_sec, end_sec)
      audio_clip.write_audiofile(output_audio_path)

def transcribe_audio(audio_filename="temp_audio.wav",  output_txt_file="transcription.txt"):
    """Transcribes the audio file using Google Web Speech API."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_filename) as source:
        audio = recognizer.record(source)

    try:
        transcript = recognizer.recognize_google(audio)
        print("Transcript:")
        print(transcript)

        # Save transcript to a text file
        with open(output_txt_file, "w") as f:
            f.write(transcript)
        print(f"Transcript saved to {output_txt_file}")

        return transcript
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")

# --- Usage ---

# Upload the video file
# uploaded = files.upload()
# video_file = list(uploaded.keys())[0]
video_file = "/content/videoplayback.mp4"

# Extract audio from the video segment (adjust start and end times as needed)
extract_audio_from_video(video_file, 407.5, 519.6)

# Transcribe the extracted audio
transcribe_audio()

# Q10


In [ ]:
from PIL import Image

# Load the scrambled image
scrambled_image = Image.open('jigsaw.webp')  # Replace with your file path if needed

# Define the mapping data
mapping_data = [
    (2, 1, 0, 0), (1, 1, 0, 1), (4, 1, 0, 2), (0, 3, 0, 3), (0, 1, 0, 4),
    (1, 4, 1, 0), (2, 0, 1, 1), (2, 4, 1, 2), (4, 2, 1, 3), (2, 2, 1, 4),
    (0, 0, 2, 0), (3, 2, 2, 1), (4, 3, 2, 2), (3, 0, 2, 3), (3, 4, 2, 4),
    (1, 0, 3, 0), (2, 3, 3, 1), (3, 3, 3, 2), (4, 4, 3, 3), (0, 2, 3, 4),
    (3, 1, 4, 0), (1, 2, 4, 1), (1, 3, 4, 2), (0, 4, 4, 3), (4, 0, 4, 4)
]

# Create a blank image for the reconstructed result
reconstructed_image = Image.new('RGB', (500, 500))

# Loop through each mapping and place pieces in their original positions
piece_size = scrambled_image.width // 5  # Each piece is assumed to be square
for original_row, original_col, scrambled_row, scrambled_col in mapping_data:
    # Calculate coordinates of the scrambled piece
    left = scrambled_col * piece_size
    upper = scrambled_row * piece_size
    right = left + piece_size
    lower = upper + piece_size

    # Crop the piece from the scrambled image
    piece = scrambled_image.crop((left, upper, right, lower))

    # Calculate coordinates for placing the piece in the reconstructed image
    dest_left = original_col * piece_size
    dest_upper = original_row * piece_size

    # Paste the piece into its correct position
    reconstructed_image.paste(piece, (dest_left, dest_upper))

# Save the reconstructed image as PNG
reconstructed_image.save('reconstructed_image.png')
print("Reconstructed image saved as 'reconstructed_image.png'")